In [1]:
import pandas as pd
from collections import Counter
from datetime import timedelta,time,datetime
import numpy as np
import copy
from sklearn.utils import shuffle
import itertools
import copy
import datetime as DateTime
import torch
import torch.nn as nn
import torch.nn.functional as F
import pandas as pd
from datetime import timedelta,time,datetime
from sklearn.utils import shuffle
import numpy as np
import pickle
from environment import *
from utilities import *
from reinforcement_learning import *

# Functionalities

In [2]:
def test_performance(env,validationSet,testSet,strategy,reinforcementAgent,e,setup):
    print("testing on validation set")
    nenv=Environment(validationSet,demand=env.demandDf,prices=None,intraPrices=None,trainDf=None)
    chargedTest,capacity=run(nenv,1,strategy,verbose=0,status='test',setup=setup,saveFiles=reinforcementAgent.saveFiles)
    print("charged on validation set ",chargedTest)
    print("testing on test set")
    nenv=Environment(testSet,demand=env.demandDf,prices=None,intraPrices=None,trainDf=None)
    testReward,capacityTest=run(nenv,1,strategy,verbose=0,status='test',setup=setup,saveFiles=reinforcementAgent.saveFiles)
    print("charged on test set ",testReward)
    fileName="val:"+str(np.round(chargedTest))+"test:"+str(np.round(testReward))
    if strategy=='RL_AC':
        reinforcementAgent.save_actor(e,fileName)
    else:
        reinforcementAgent.save_critic(e,fileName)

In [3]:
def run(env,episodes,strategy,validationSet=None,testSet=None,verbose=1,status='test',setup='setup1',
        saveFiles=None,hp=None):
    totalCharged=0
    totalEnergy=0
    intervalLength=env.intervalLength
    
    if "RL" in strategy:
        if type(hp)!=type(None):
            reinforcementAgent=ReinforcementLearning(gamma=hp['gamma'],decay=hp['decay'],
                                                 setup=setup,learnRate=hp['learnRate'],capacity=hp['capacity'],
                                                 tau=hp['tau'],policyBatchSize=hp['policyBatchSize'],
                                                saveFiles=saveFiles,saveNorm=hp['saveNormalizerAfter'],
                                                qUpdateFreq=hp['qUpdateFreq'],updatePol=hp['updatePolicyAfter'])
        else:
            reinforcementAgent=ReinforcementLearning(saveFiles=saveFiles,setup=setup) 

        
    #take steps untill env return False
    ite=-1
    for e in range(episodes):
        dayCharged=0
        pendingJobs=[]
        
        #start with first job
        jobs,done,time=env.step()
        for job in jobs:
            pendingJobs.append(job)
        
        done=False
        pendingJobs,noUnfeasible,leftover=env.get_feasible_jobs(pendingJobs,time)
        
            
        ####change to 50
        if e>=1 and (e-1)%hyperparam['saveEvery']==0 and status!='test' and "RL" in strategy:
             test_performance(env,validationSet,testSet,strategy,reinforcementAgent,e,setup)
            
        episodeStep=-1
        while not done:
            episodeStep+=1
            ite+=1
            if verbose==2:
                print("iteraction no: ",ite,"out of: ",str(12*24)) 
         
            if strategy=='priority':
                pendingJobs=sorted(pendingJobs, key=lambda tup: tup['duetime_no'])
                
            if strategy=='arrival':
                pendingJobs=sorted(pendingJobs, key=lambda tup: tup['arrivaltime_no'])
                
            if strategy=='opt_disp':
                pendingJobs=sorted(pendingJobs, key=lambda tup: (-max(0,(tup['TotalEnergy(KWh)']/tup['MaxPower(KW)'])-((tup['duetime_no']-time)/12)),tup['duetime_no']))
            
            if strategy=='RL_AC':
                pendingJobs=reinforcementAgent.actor_critic_policy(pendingJobs,time,intervalLength,ite,e,episodeStep,status)
            if strategy=="RL_DQN":
                pendingJobs=reinforcementAgent.dqn_policy(env,pendingJobs,time,intervalLength,ite,status)
                
            #plan jobs untill all jobs are planned or there is no more capacity
            pendingJobs,charged=env.process_queue(time,pendingJobs)
            dayCharged+=charged
            
            #take next step
            jobs,done,time=env.step()
            for job in jobs:
                pendingJobs.append(job)
            pendingJobs,noUnfeasible,leftover=env.get_feasible_jobs(pendingJobs,time)
            
        totalCharged+=dayCharged
        totalDem=int(totalDemand(env.demand,intervalLength))
        if verbose==1:
            print("losses:_",np.mean(reinforcementAgent.losses))
            print("episode:__",str(e),"orig demand___",totalDem,"total  charged___",int(dayCharged))

        if done:
            env.reset()

    return totalCharged,env.demandLeft
    
    
    

# Set parameters

In [4]:
dataFiles={
    'chargeSes':"data sets/preproc_charge_sessions.csv",
    'source':"source_solution/source_orig_0_95"
}
saveFiles={'critic':'output files/critic',
           'target':'output files/target',
           'actor':'output files/actor',
           'normalizer':'output files/normalizer.p'
}
setup='setup1'

In [5]:
intervalLength=5
surplusName="surplus 105"
numberOfEpisodes=15

In [6]:
hyperparam={
        'gamma':0.95,
        'decay':30000,
        'learnRate':0.0001,
        'capacity':30000,
        'tau':4000,
        'policyBatchSize':5,
        'saveEvery':7, ##set to 50
        'qUpdateFreq':2,
        'updatePolicyAfter':2, ###set to 2000
        'saveNormalizerAfter':2016
}

# Read data and split test/train

In [7]:
demand1= pickle.load( open(dataFiles['source'], "rb" ))
df = pd.read_csv(dataFiles['chargeSes'],parse_dates=["Started", "Ended",'start_time','EndCharge','arrival_time', 'due_time','latest_start_time'])
trainDf=df[df['days_for_aggregation']<28]
validationDf=df[df['days_for_aggregation'].isin([28,29,30,31,32,33,34])]
testDf=df[df['days_for_aggregation'].isin([35,36,37,38,39,40,41])]
trainDf['target']=0



C:\Users\larsr\Anaconda3\lib\site-packages\ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


# Train algorithm


In [8]:
env=Environment(trainDf,demand=demand1,prices=None,intraPrices=None,trainDf=None)
uncharged,capacity=run(env,numberOfEpisodes,"RL_DQN",validationSet=validationDf,testSet=testDf,verbose=1,status='train',
                       hp=hyperparam,setup=setup,saveFiles=saveFiles)

KeyboardInterrupt: 

In [ ]:

env=Environment(trainDf,demand=demand1,prices=None,intraPrices=None,trainDf=None)
uncharged,capacity=run(env,numberOfEpisodes,"RL_AC",validationSet=validationDf,testSet=testDf,verbose=1,status='train',
                       saveFiles=saveFiles,hp=hyperparam,setup=setup)